In [105]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import _pickle as cPickle
import seaborn as sb
%matplotlib inline
pd.set_option('display.max_rows',999)
pd.set_option('display.max_columns',999)

In [106]:
df = pd.read_pickle('dfaccept')

In [107]:
df = df[['loan_status','issue_d','last_pymnt_d','loan_amnt','int_rate','term','out_prncp','installment','total_pymnt']]

In [108]:
df.head(5)

,loan_status,issue_d,last_pymnt_d,loan_amnt,int_rate,term,out_prncp,installment,total_pymnt
0,Fully Paid,Dec-2015,Jan-2019,3600.0,13.99,36 months,0.00,123.03,4421.723917
1,Fully Paid,Dec-2015,Jun-2016,24700.0,11.99,36 months,0.00,820.28,25679.660000
2,Fully Paid,Dec-2015,Jun-2017,20000.0,10.78,60 months,0.00,432.66,22705.924294
3,Current,Dec-2015,Feb-2019,35000.0,14.85,60 months,15897.65,829.90,31464.010000
4,Fully Paid,Dec-2015,Jul-2016,10400.0,22.45,60 months,0.00,289.91,11740.500000


In [109]:
df_negClass = df[df['loan_status'].isin(['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid'])]

df_posClass = df[df['loan_status'].isin(['Charged Off', 
                                         'Does not meet the credit policy. Status:Charged Off', 'Default'])]


df = pd.concat([df_negClass, df_posClass]).reset_index(drop = True)

In [110]:
df['loan_status_binary'] = (~df.loan_status.isin(['Fully Paid',\
                                                  'Does not meet the credit policy. Status:Fully Paid'])).astype(int)

In [111]:
df.loan_status_binary.value_counts()

0    1078739
1     269360
Name: loan_status_binary, dtype: int64

In [112]:
df.drop('loan_status', axis = 1, inplace = True)

In [113]:
df.head(5)

,issue_d,last_pymnt_d,loan_amnt,int_rate,term,out_prncp,installment,total_pymnt,loan_status_binary
0,Dec-2015,Jan-2019,3600.0,13.99,36 months,0.0,123.03,4421.723917,0
1,Dec-2015,Jun-2016,24700.0,11.99,36 months,0.0,820.28,25679.660000,0
2,Dec-2015,Jun-2017,20000.0,10.78,60 months,0.0,432.66,22705.924294,0
3,Dec-2015,Jul-2016,10400.0,22.45,60 months,0.0,289.91,11740.500000,0
4,Dec-2015,May-2017,11950.0,13.44,36 months,0.0,405.18,13708.948530,0


In [114]:
df.shape

(1348099, 9)

In [115]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1348099 entries, 0 to 1348098
Data columns (total 9 columns):
issue_d               1348099 non-null object
last_pymnt_d          1345774 non-null object
loan_amnt             1348099 non-null float64
int_rate              1348099 non-null float64
term                  1348099 non-null object
out_prncp             1348099 non-null float64
installment           1348099 non-null float64
total_pymnt           1348099 non-null float64
loan_status_binary    1348099 non-null int32
dtypes: float64(5), int32(1), object(3)
memory usage: 87.4+ MB


In [83]:
df.term=df.term.str.extract('(\d+)') #extract numeric number from term, removing ' months' string

In [84]:
df['term'] = pd.to_numeric(df['term'])

In [85]:
df.head(5)

,issue_d,last_pymnt_d,loan_amnt,int_rate,term,out_prncp,installment,total_pymnt,loan_status_binary
0,Dec-2015,Jan-2019,3600.0,13.99,36,0.0,123.03,4421.723917,0
1,Dec-2015,Jun-2016,24700.0,11.99,36,0.0,820.28,25679.660000,0
2,Dec-2015,Jun-2017,20000.0,10.78,60,0.0,432.66,22705.924294,0
3,Dec-2015,Jul-2016,10400.0,22.45,60,0.0,289.91,11740.500000,0
4,Dec-2015,May-2017,11950.0,13.44,36,0.0,405.18,13708.948530,0


In [86]:
df.term.max()

60

In [87]:
df.term.min()

36

In [88]:
def missing_data(df):
    miss = np.sum(df.isnull(),axis =0)
    miss = miss.loc[miss>0].sort_values(ascending = False)
    missmean = np.mean(df.isnull(),axis =0)
    missmean = missmean.loc[missmean>0].sort_values(ascending = False)
    return pd.concat([miss.rename('count'),missmean.rename('mean')],axis = 1)

In [89]:
missing_data(df)

,count,mean
last_pymnt_d,2325,0.001725


In [22]:
#df=df.dropna() #remove missing payment date rows

In [90]:
df.shape

(1348099, 9)

In [91]:
missing_data(df)

,count,mean
last_pymnt_d,2325,0.001725


In [116]:
df["interest_over_principal"]=df.total_pymnt-df.loan_amnt

In [117]:
df.head(5)

,issue_d,last_pymnt_d,loan_amnt,int_rate,term,out_prncp,installment,total_pymnt,loan_status_binary,interest_over_principal
0,Dec-2015,Jan-2019,3600.0,13.99,36 months,0.0,123.03,4421.723917,0,821.723917
1,Dec-2015,Jun-2016,24700.0,11.99,36 months,0.0,820.28,25679.660000,0,979.660000
2,Dec-2015,Jun-2017,20000.0,10.78,60 months,0.0,432.66,22705.924294,0,2705.924294
3,Dec-2015,Jul-2016,10400.0,22.45,60 months,0.0,289.91,11740.500000,0,1340.500000
4,Dec-2015,May-2017,11950.0,13.44,36 months,0.0,405.18,13708.948530,0,1758.948530


In [118]:
df.interest_over_principal=df.interest_over_principal.round(2)

In [95]:
df.head(5)

,issue_d,last_pymnt_d,loan_amnt,int_rate,term,out_prncp,installment,total_pymnt,loan_status_binary,interest_over_principal
0,Dec-2015,Jan-2019,3600.0,13.99,36,0.0,123.03,4421.723917,0,821.72
1,Dec-2015,Jun-2016,24700.0,11.99,36,0.0,820.28,25679.660000,0,979.66
2,Dec-2015,Jun-2017,20000.0,10.78,60,0.0,432.66,22705.924294,0,2705.92
3,Dec-2015,Jul-2016,10400.0,22.45,60,0.0,289.91,11740.500000,0,1340.50
4,Dec-2015,May-2017,11950.0,13.44,36,0.0,405.18,13708.948530,0,1758.95


In [119]:
df_paid = df[df['loan_status_binary'] == 0]
df_default = df[df['loan_status_binary']== 1]

In [97]:
df_paid.head(5)

,issue_d,last_pymnt_d,loan_amnt,int_rate,term,out_prncp,installment,total_pymnt,loan_status_binary,interest_over_principal
0,Dec-2015,Jan-2019,3600.0,13.99,36,0.0,123.03,4421.723917,0,821.72
1,Dec-2015,Jun-2016,24700.0,11.99,36,0.0,820.28,25679.660000,0,979.66
2,Dec-2015,Jun-2017,20000.0,10.78,60,0.0,432.66,22705.924294,0,2705.92
3,Dec-2015,Jul-2016,10400.0,22.45,60,0.0,289.91,11740.500000,0,1340.50
4,Dec-2015,May-2017,11950.0,13.44,36,0.0,405.18,13708.948530,0,1758.95


In [98]:
df_default.head(5)

,issue_d,last_pymnt_d,loan_amnt,int_rate,term,out_prncp,installment,total_pymnt,loan_status_binary,interest_over_principal
1078739,Dec-2015,May-2017,18000.0,19.48,60,0.0,471.70,9452.74,1,-8547.26
1078740,Dec-2015,Apr-2017,4225.0,14.85,36,0.0,146.16,2558.87,1,-1666.13
1078741,Dec-2015,Nov-2018,16000.0,12.88,36,0.0,538.18,17396.63,1,1396.63
1078742,Dec-2015,Jul-2016,24250.0,24.24,60,0.0,701.01,4124.42,1,-20125.58
1078743,Dec-2015,Feb-2018,25000.0,13.99,60,0.0,581.58,14490.92,1,-10509.08


In [121]:
df.interest_over_principal.sum()

542888143.7200009

In [122]:
df.loan_amnt.sum()

19424757025.0

In [124]:
(542888143.7200009/19424757025.0)*100

2.7948259173656296

In [120]:
df_paid.shape

(1078739, 10)

In [61]:
df_default.shape

(269360, 10)

In [69]:
df_paid.interest_over_principal.mean() #model scoring top left

2317.7083760019423

In [59]:
df_default.interest_over_principal.mean() #model scoring bottom left

-7266.53650170767

In [70]:
df_paid.loan_amnt.mean()

14124.637238479372

In [71]:
df_default.loan_amnt.mean()

15547.816955004455

In [99]:
df.interest_over_principal.mean()

402.7064360406599

In [100]:
df.loan_amnt.mean()

14408.998912542773

In [25]:
df['issue_d'] = df['issue_d'].apply(pd.to_datetime)

In [ ]:
df['last_pymnt_d'] = df['last_pymnt_d'].apply(pd.to_datetime)